### Intended as a script which generates a .cvs report in one run

In [1]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.preprocessing import StandardScaler;
from sklearn.metrics import mean_squared_error;

2.3.0


In [2]:
pd.set_option('display.max_columns', 300);
pd.set_option('display.max_rows', 300);
tf.keras.backend.set_floatx('float64');

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[]

In [4]:
MONTHS = 72;
SPLIT = 60; # 2014-2018: training, 2019: testing.
# BATCH_SIZE = 19; # used in NN_v1
BATCH_SIZE = 30;
WINDOW_SIZE = 3;

TEST_LENGTH = MONTHS - SPLIT;

### Data preparation

In [5]:
multi_data = pd.read_csv('../data/zri_acs_bikeshare_merged.csv', index_col = 0);
zip_ids = multi_data.index.unique();

multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "datetime"],\
                 axis = 1, inplace = True);

multi_data.head()

,zri,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,percent_65+,percent_rental_units_vacant,percent_rental_units_occupied,percent_graduate_deg,percent_bachelors,percent_associates,percent_highschool,percent_less_highschool,percent_commute_public_transport,percent_commute_less_30,percent_buildings_less_10_units,percent_buildings_10_19_units,percent_buildings_20_49_units,percent_buildings_50+_units,percent_commute_30_to_59,percent_commute_60_to_89,percent_commute_90_more,percent_new_city,percent_new_unit,percent_units_owner_occupied,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1013,928.0,0.728315,0.020951,0.013732,0.219830,0.0,0.004959,0.230010,0.311464,0.309663,0.148864,0.068131,0.931869,0.028046,0.056892,0.259500,0.384872,0.225671,0.023499,0.741732,0.890613,0.030177,0.034856,0.044355,0.227721,0.017344,0.013031,0.108543,0.040940,0.443651,62.0,20475.25,0.191440,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
1013,931.0,0.727052,0.021179,0.013708,0.220867,0.0,0.004917,0.229526,0.311552,0.310406,0.148516,0.067207,0.932793,0.028179,0.057377,0.259450,0.384586,0.225073,0.023365,0.740587,0.889943,0.030154,0.035083,0.044820,0.227757,0.017701,0.013401,0.108828,0.040926,0.441963,62.0,20517.50,0.192359,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
1013,934.0,0.725788,0.021408,0.013685,0.221904,0.0,0.004874,0.229041,0.311641,0.311150,0.148168,0.066283,0.933717,0.028313,0.057863,0.259400,0.384301,0.224475,0.023230,0.739441,0.889273,0.030131,0.035311,0.045285,0.227792,0.018058,0.013771,0.109112,0.040912,0.440275,62.0,20559.75,0.193277,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
1013,929.0,0.724525,0.021636,0.013661,0.222940,0.0,0.004832,0.228557,0.311729,0.311894,0.147820,0.065359,0.934641,0.028446,0.058348,0.259350,0.384016,0.223877,0.023096,0.738295,0.888603,0.030108,0.035538,0.045750,0.227828,0.018415,0.014142,0.109397,0.040897,0.438587,62.0,20602.00,0.194196,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
1013,929.0,0.723261,0.021864,0.013638,0.223977,0.0,0.004789,0.228072,0.311818,0.312638,0.147472,0.064435,0.935565,0.028579,0.058833,0.259299,0.383730,0.223279,0.022961,0.737149,0.887934,0.030086,0.035766,0.046215,0.227864,0.018772,0.014512,0.109682,0.040883,0.436898,62.0,20644.25,0.195114,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0


In [6]:
FEATURES = multi_data.shape[1] - 1;

In [7]:
# for x, y in dataset_train:
#     print(np.array(x), np.array(y))

### Utility functions

In [8]:
@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series); #(43,)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True);
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)); #(13,43)
    dataset = dataset.shuffle(shuffle_buffer)\
                     .map(lambda window: (window[:-1, 1:], window[-1][0]));
    dataset = dataset.batch(batch_size).prefetch(1);
    return dataset;

In [9]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time Frame")
    plt.ylabel("ZRI")
    plt.grid(True)

### Neural network center

In [10]:
def NN_model(dataset, termination, test = None):
    class myCallbacks(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            mse = logs.get("mse");
            if(mse < termination):
                print("\nGot an mse at {:.4f} in round {} and stopped training\n".format(mse, epoch));
                self.model.stop_training = True;
            
    tf.keras.backend.clear_session();
    # dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    callback = myCallbacks();
    
    model = tf.keras.models.Sequential([
#         tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
#                           input_shape=[None]),
#       tf.keras.layers.Conv1D(filters=32, kernel_size=3,
#                           strides=1, padding="causal",
#                           activation="relu",
#                           input_shape=[None, WINDOW_SIZE, FEATURES+1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True, input_shape = [None, WINDOW_SIZE, FEATURES])),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        # tf.keras.layers.SimpleRNN(8, return_sequences=True),
        #  tf.keras.layers.SimpleRNN(16, return_sequences=True),
        tf.keras.layers.Dense(8, activation="relu"),
    #   tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
        # tf.keras.layers.Lambda(lambda x: x * 2.0)
    ]);
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer="adam",
                  metrics=["mae", "mse"])

    model.build((None,WINDOW_SIZE,FEATURES))
    # model.summary()

    if not test: history = model.fit(dataset, epochs=500, callbacks = [callback], verbose = 0);
    else: history = model.fit(dataset, epochs=500, validation_data=test,\
                              callbacks=[callback], verbose = 0);
    return model;

In [11]:
def NN_forecast(model, series_transformed):
    forecast = []
    results = []
    for time in range(MONTHS - WINDOW_SIZE):
        forecast.append(model.predict(series_transformed[np.newaxis, time:time + WINDOW_SIZE, 1:]))

    results = [float(x[-1][0]) for x in forecast];
    actual = list(series_transformed[WINDOW_SIZE:, 0]);
    
    return results, actual; #, pure_forecast;

In [12]:
@tf.autograph.experimental.do_not_convert
def NN_test(ZONE, termination=0, plot=False):
    '''
    Input: ZONE
    Output: the RMSE of a NN model on the predicted train, partially predicted test, and complete predicted test.
    '''
    # Collection of data
    series_frame = multi_data[multi_data.index == ZONE];
    # series_frame.sort_values("datetime", ascending = True, inplace = True);
    
    # Standardization
    scaler = StandardScaler();
    series_transformed = scaler.fit_transform(series_frame);
    
    # Train test split
    series_train = series_transformed[:SPLIT];
    series_test = series_transformed[SPLIT-WINDOW_SIZE:];
    
    # Window the training set to make input of the NN
    dataset_train = windowed_dataset(series_train, WINDOW_SIZE, BATCH_SIZE, 60);
    # dataset_test = windowed_dataset(series_test, WINDOW_SIZE, BATCH_SIZE, 60);
    
    model = NN_model(dataset_train, termination);
    
    time_train = list(range(SPLIT));
    time_test = list(range(SPLIT, MONTHS));
    
    # Forecasting
    # results, actual, pure_forecast = NN_forecast(model, series_transformed);
    results, actual = NN_forecast(model, series_transformed);
    
    # Compute MSE
    MSE_train = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5 * scaler.var_[0]**0.5;
    MSE_pure = mean_squared_error(actual[-TEST_LENGTH:], results[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    # MSE_pure = mean_squared_error(actual[-TEST_LENGTH:], pure_forecast[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    
    if plot: # If the plot option is selected, plot the graph.
        time_actual = range(WINDOW_SIZE, MONTHS);
        plt.figure(figsize=(10, 6))
        plot_series(time_actual, actual);
        plot_series(time_actual, results);
        # plot_series(time_test, pure_forecast);
        plt.show();

    return MSE_train, MSE_pure,\
        np.array(results[-TEST_LENGTH:]) * scaler.var_[0]**0.5 + scaler.mean_[0];

### The script

In [13]:
score_dict = {"zip":[], "RMSE_train":[], "RMSE_pure":[], "forecast":[]};
with tf.device('/cpu:0'):
    for ZONE in zip_ids:
        %time M_train, M_pure, forecast = NN_test(ZONE, termination=0.005, plot=False);
        print(ZONE, M_train, M_pure, forecast);

        score_dict["zip"].append(ZONE);
        score_dict["RMSE_train"].append(M_train);
        # score_dict["RMSE_test"].append(M_test);
        score_dict["RMSE_pure"].append(M_pure);
        score_dict["forecast"].append(forecast);


Got an mse at 0.0050 in round 55 and stopped training

Wall time: 8.49 s
1013 5.348199444608841 37.520381250853866 [1102.40167748 1097.29432132 1095.49071862 1098.69995718 1101.44091355
 1103.67181521 1105.37525047 1106.5385583  1106.35649147 1105.7768051
 1104.84946598 1103.63303293]

Got an mse at 0.0050 in round 84 and stopped training

Wall time: 8.48 s
1020 5.1031316219384095 84.3254014312301 [1122.24455748 1120.16733536 1117.98532271 1117.27462832 1114.3499905
 1107.08657202 1092.0237032  1080.86468499 1076.24539459 1074.540275
 1073.98499537 1073.95506003]

Got an mse at 0.0050 in round 239 and stopped training

Wall time: 12.4 s
1040 4.243376652220285 51.87286643909586 [1097.27676502 1097.37901189 1102.99531829 1101.93420458 1099.29748389
 1095.734547   1091.15049628 1084.9663154  1078.25915925 1071.462616
 1065.25532317 1060.37910846]

Got an mse at 0.0050 in round 205 and stopped training

Wall time: 11.7 s
1085 4.923289113258683 25.89802175083844 [1145.47341442 1139.1254430


Got an mse at 0.0050 in round 189 and stopped training

Wall time: 12.6 s
1851 9.031958075346672 118.54287924206808 [1991.66929781 1985.63216214 1980.32531661 1978.71747362 1974.61753357
 1967.75863855 1958.28038684 1946.76250964 1920.63902189 1904.94205637
 1895.35256602 1889.47944652]

Got an mse at 0.0050 in round 246 and stopped training

Wall time: 14.1 s
1852 9.258310844839812 80.13859486065662 [1989.49955182 1986.14066488 1982.99866163 1979.13826269 1975.67527671
 1973.69259495 1971.6229585  1969.31608807 1966.76565407 1964.11348317
 1961.55453915 1959.23826806]

Got an mse at 0.0050 in round 283 and stopped training

Wall time: 14.9 s
1854 9.976865513529667 129.61094078043726 [2010.82689667 2010.00926417 2009.14192072 2007.7109757  2006.32278667
 2004.54402499 2002.39608414 1999.93335428 1997.23053092 1994.36722678
 1991.41821777 1988.45446049]

Got an mse at 0.0050 in round 152 and stopped training

Wall time: 11.8 s
1876 7.875068446488174 119.41385926354188 [2200.54891295 21

Wall time: 20.4 s
2129 8.680338785970445 144.4966545962887 [2781.8117332  2776.7903164  2773.82356468 2781.50678789 2779.07095461
 2775.84723342 2772.29900593 2768.80575042 2765.63072326 2762.92007795
 2760.71317594 2759.05428048]

Got an mse at 0.0050 in round 248 and stopped training

Wall time: 14.2 s
2130 9.862606534142675 208.26104117725868 [2608.71451612 2608.70166317 2610.66337426 2610.98556633 2611.72174693
 2614.76169412 2614.5168187  2614.27802649 2620.91972404 2628.6766995
 2638.45451419 2652.24765434]

Got an mse at 0.0049 in round 187 and stopped training

Wall time: 10.8 s
2131 10.168538418459786 222.66266296783252 [2370.25874085 2361.19958921 2363.84232764 2376.67649436 2377.88482128
 2379.14440391 2380.42732084 2381.69886343 2382.09152606 2382.19426431
 2382.2897046  2382.38064818]

Got an mse at 0.0049 in round 237 and stopped training

Wall time: 13.2 s
2132 13.272979446527122 250.5397859746403 [2365.87343456 2359.01316611 2363.00565854 2368.32017332 2373.52996971
 23


Got an mse at 0.0050 in round 100 and stopped training

Wall time: 9.2 s
2360 10.740639032741853 92.49918571889859 [1776.78806267 1769.48717799 1764.43125334 1768.54891976 1770.82794825
 1772.41612515 1775.85051887 1778.92754128 1781.43145239 1783.63866946
 1785.66559546 1787.48097079]
Wall time: 19.3 s
2445 10.03463772352908 93.7218578409254 [2767.66315745 2740.31685817 2725.58971936 2751.02118077 2736.37587864
 2725.9181985  2719.47183004 2718.82523959 2719.59851994 2720.87689865
 2722.50841494 2724.33353502]

Got an mse at 0.0050 in round 100 and stopped training

Wall time: 8.57 s
2446 7.813666181039868 108.45913033845547 [2881.39074785 2879.82028298 2881.85055553 2889.37802197 2888.31175536
 2885.0588421  2881.55264569 2878.08664889 2874.75339774 2871.5240291
 2868.34019046 2865.15133979]

Got an mse at 0.0050 in round 195 and stopped training

Wall time: 12.5 s
2451 5.256228102836566 108.36367093383107 [2411.7651351  2400.83738271 2393.36588941 2387.7195774  2382.58319104
 2379.


Got an mse at 0.0050 in round 56 and stopped training

Wall time: 8.17 s
2908 6.933712478118352 32.60146659700984 [1269.80163683 1265.69494414 1263.83271608 1265.18797411 1266.53736202
 1267.50456657 1267.20531131 1265.72697267 1262.48964099 1258.17176962
 1253.48913426 1248.60840949]

Got an mse at 0.0050 in round 128 and stopped training

Wall time: 9.43 s
2909 6.464648851322396 66.78330234138213 [1244.78928369 1245.36427011 1245.6046912  1245.39159011 1243.98215789
 1240.50484821 1233.31813441 1220.56332713 1202.66441353 1185.73644426
 1176.10903698 1172.45771707]

Got an mse at 0.0050 in round 129 and stopped training

Wall time: 10.5 s
2910 5.054830100836267 43.76757601210747 [1210.93874501 1211.36036937 1210.81903817 1210.03086523 1208.38211771
 1206.42759004 1201.95718138 1197.68530972 1194.01615573 1190.52780208
 1186.95777581 1182.17055448]

Got an mse at 0.0050 in round 146 and stopped training

Wall time: 10.6 s
2914 5.789727970909733 72.52131283424615 [1258.33337198 1255.5

...


Got an mse at 0.0050 in round 74 and stopped training

Wall time: 6.14 s
92626 7.66292119143871 66.48818329577718 [2201.01024693 2199.52809698 2198.1402711  2191.05837419 2182.6372927
 2173.58501352 2164.63632311 2155.84571786 2145.93006296 2140.33468923
 2138.20932617 2136.02779124]

Got an mse at 0.0050 in round 137 and stopped training

Wall time: 7.2 s
92627 9.65591790525254 34.23574341447738 [2293.65530428 2292.06985579 2290.08381067 2289.90616011 2290.96498812
 2291.20748931 2291.9120118  2288.85252646 2282.72530581 2275.65347045
 2267.81613455 2258.76800993]

Got an mse at 0.0050 in round 68 and stopped training

Wall time: 6.36 s
92647 9.912506079015559 100.61353696663619 [2106.99347574 2104.94580322 2103.86277495 2108.61882813 2114.01888421
 2120.83741633 2129.89043595 2141.02910241 2152.14978686 2159.27621236
 2158.69039809 2156.06669598]

Got an mse at 0.0050 in round 75 and stopped training

Wall time: 6.11 s
92648 15.205149599196803 60.60955430759332 [2325.75699918 2327.7


Got an mse at 0.0049 in round 113 and stopped training

Wall time: 6.91 s
94025 13.817766496770464 100.38543397726099 [3363.29630374 3375.33855818 3385.46820799 3386.77856887 3386.74304189
 3385.47128247 3383.79635058 3380.98701308 3376.15543201 3370.02299896
 3363.59909771 3356.97595254]

Got an mse at 0.0050 in round 131 and stopped training

Wall time: 7.39 s
94040 15.91593341472231 107.07080293783916 [3395.70615242 3396.85493509 3398.10634086 3392.30369332 3386.06126655
 3379.43596027 3373.17780157 3367.42563503 3361.08643806 3350.96689176
 3328.10747819 3299.61951738]

Got an mse at 0.0049 in round 42 and stopped training

Wall time: 5.71 s
94061 18.902037306021594 152.20211022072587 [3044.08563016 3045.29138038 3044.73784043 3038.0036772  3029.39431609
 3019.85348573 3017.58654699 3012.99622245 3006.0935371  2996.15884036
 2975.17373728 2950.87666597]
Wall time: 14.2 s
94066 75.98055573982634 58.04279132125321 [2830.67411633 2824.77286129 2818.34560581 2811.71298792 2803.3074431


Got an mse at 0.0050 in round 134 and stopped training

Wall time: 7.12 s
94801 14.514630358572292 87.1807542637478 [2238.14728372 2238.88999465 2238.50323451 2228.58878735 2216.88943203
 2203.43838981 2189.32153794 2177.57072337 2169.88274294 2164.36155036
 2159.70645633 2158.34637237]

Got an mse at 0.0049 in round 78 and stopped training

Wall time: 6.52 s
94806 18.043286332130382 39.173777973310784 [2119.54714501 2124.55938598 2134.91346481 2146.42206436 2153.8262565
 2157.00619359 2157.707229   2157.28241383 2150.92277726 2143.85860706
 2136.09184259 2127.76906606]

Got an mse at 0.0049 in round 41 and stopped training

Wall time: 5.38 s
94901 13.990234216066897 77.02615527843574 [2656.00781496 2665.32683494 2667.58811458 2653.78854921 2640.28554287
 2626.27940996 2610.77254203 2593.0048799  2573.20532063 2553.11747548
 2535.11499878 2519.5202797 ]

Got an mse at 0.0050 in round 110 and stopped training

Wall time: 7.06 s
94903 13.881040527848258 146.0936934734718 [2570.54425331 


Got an mse at 0.0050 in round 145 and stopped training

Wall time: 7.72 s
97214 6.579790087654549 46.8169371879121 [1577.18795427 1577.41758932 1577.54579107 1576.15225171 1574.8735805
 1573.70786461 1572.62314217 1571.58774434 1570.55939369 1569.46980862
 1568.19560979 1566.50492497]

Got an mse at 0.0050 in round 77 and stopped training

Wall time: 6.18 s
97229 8.419026916848221 65.36952646784054 [1434.87067474 1433.47482628 1428.22437089 1421.0675359  1414.12567006
 1408.30867767 1404.71874868 1404.61196603 1406.57930711 1406.09751543
 1406.49206552 1406.17123659]

Got an mse at 0.0050 in round 213 and stopped training

Wall time: 8.69 s
97239 8.587089330198019 38.63328429031837 [1598.08599245 1594.75595328 1591.25788298 1597.213289   1598.690638
 1599.56066999 1600.32333423 1600.97861309 1601.50867847 1601.88423113
 1602.07277246 1602.04582865]

Got an mse at 0.0050 in round 221 and stopped training

Wall time: 9.19 s
97401 7.257623928944963 103.1898012430911 [1266.31977328 1259.8


Got an mse at 0.0050 in round 97 and stopped training

Wall time: 6.55 s
98409 9.462015346798278 39.63425516801345 [1266.50341069 1269.68061713 1272.09219555 1271.52094447 1270.27766686
 1268.36678016 1265.81057007 1262.65416478 1258.96410212 1254.81939836
 1250.29855011 1246.89375268]

Got an mse at 0.0049 in round 42 and stopped training

Wall time: 5.54 s
98444 8.923549965680944 66.51349724776576 [1229.96470681 1231.2714357  1231.3432518  1226.80844006 1221.80634386
 1216.19540838 1209.92612636 1203.1265311  1196.11801952 1189.3001922
 1182.66575596 1176.40928507]

Got an mse at 0.0050 in round 84 and stopped training

Wall time: 6.26 s
98466 10.054895571791025 62.590897826254796 [1393.73097334 1398.36447998 1402.0939617  1395.59728502 1386.24613093
 1374.0016441  1359.50931472 1344.03098524 1328.78497791 1314.64551344
 1301.98804758 1292.58113055]

Got an mse at 0.0050 in round 101 and stopped training

Wall time: 6.72 s
98498 10.284659674787449 124.53003526076614 [1376.35602316 1

In [14]:
print(len(score_dict["RMSE_pure"]),\
      sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]));

rmse = sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]);

1301 62.3814968625838


In [15]:
test = pd.DataFrame(score_dict);
test.head()

,zip,RMSE_train,RMSE_pure,forecast
0,1013,5.348199,37.520381,"[1102.4016774826903, 1097.294321323687, 1095.4..."
1,1020,5.103132,84.325401,"[1122.244557476922, 1120.1673353589629, 1117.9..."
2,1040,4.243377,51.872866,"[1097.2767650224878, 1097.3790118863271, 1102...."
3,1085,4.923289,25.898022,"[1145.4734144236827, 1139.1254430259112, 1136...."
4,1104,5.161220,49.803714,"[1116.170896600122, 1118.0402318837153, 1119.2..."


In [16]:
test.to_csv('NN_featureonly_window_1_overfit_{:.2f}.csv'.format(rmse), index = False)